In [1]:
!pip install --upgrade pylda2vec
!pip install --upgrade chainer
!pip install --upgrade spacy
!python -m spacy download en_core_web_md
!pip install --upgrade jellyfish
!pip install --upgrade gensim

Requirement already up-to-date: chainer in /usr/local/lib/python3.6/dist-packages (6.5.0)
     |████████████████████████████████| 10.3MB 6.2MB/s 
     |████████████████████████████████| 3.7MB 37.4MB/s 
     |████████████████████████████████| 2.2MB 46.5MB/s 
     |████████████████████████████████| 122kB 45.2MB/s 
  Found existing installation: blis 0.2.4
    Uninstalling blis-0.2.4:
      Successfully uninstalled blis-0.2.4
  Found existing installation: preshed 2.0.1
    Uninstalling preshed-2.0.1:
      Successfully uninstalled preshed-2.0.1
  Found existing installation: thinc 7.0.8
    Uninstalling thinc-7.0.8:
      Successfully uninstalled thinc-7.0.8
  Found existing installation: spacy 2.1.9
    Uninstalling spacy-2.1.9:
      Successfully uninstalled spacy-2.1.9
     |████████████████████████████████| 96.4MB 43.9MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.0-cp36-none-any.whl size=98072933 sha256=af8c755a77f330cabf682a95d7773ff8bfa9855028f19bc17ecb570dd

In [0]:
from os import path
import pickle
with open(path.expanduser('./drive/My Drive/data.pkl'), "rb") as input_file:
    data1 = pickle.load(input_file)

In [0]:
# Author: Chris Moody <chrisemoody@gmail.com>
# License: MIT

# This simple example loads the newsgroups data from sklearn
# and train an LDA-like model on it
import logging
import pickle

import numpy as np

from lda2vec import preprocess, Corpus

logging.basicConfig()


# Preprocess data
max_length = data1['transcript'].apply(lambda x: len(x.split(' '))).max()   # Limit of words per document

tokens, vocab = preprocess.tokenize(data1['transcript'].values.tolist(), max_length, merge=False)
corpus = Corpus()
# Make a ranked list of rare vs frequent words
corpus.update_word_count(tokens)
corpus.finalize()
# The tokenization uses spaCy indices, and so may have gaps
# between indices for words that aren't present in our dataset.
# This builds a new compact index
compact = corpus.to_compact(tokens)
# Remove extremely rare words
pruned = corpus.filter_count(compact, min_count=2)
# Convert the compactified arrays into bag of words arrays
bow = corpus.compact_to_bow(pruned)
# Words tend to have power law frequency, so selectively
# downsample the most prevalent words
clean = corpus.subsample_frequent(pruned)
# Now flatten a 2D array of document per row and word position
# per column to a 1D array of words. This will also remove skips
# and OoV words
doc_ids = np.arange(pruned.shape[0])
flattened, (doc_ids,) = corpus.compact_to_flat(pruned, doc_ids)
assert flattened.min() >= 0
# Fill in the pretrained word vectors
n_dim = 300
fn_wordvc = './drive/My Drive/GoogleNews-vectors-negative300.bin'
vectors, s, f = corpus.compact_word_vectors(vocab, filename=fn_wordvc)


In [0]:
# Save all of the preprocessed files
with open('vocab.pkl', 'wb') as handle:
    pickle.dump(vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('corpus.pkl', 'wb') as handle:
    pickle.dump(corpus, handle, protocol=pickle.HIGHEST_PROTOCOL)

np.save("flattened", flattened)
np.save("doc_ids", doc_ids)
np.save("pruned", pruned)
np.save("bow", bow)
np.save("vectors", vectors)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
! cp ./* drive/My\ Drive

cp: -r not specified; omitting directory './drive'
cp: -r not specified; omitting directory './sample_data'


In [3]:
!curl https://colab.chainer.org/install | sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0   6245      0 --:--:-- --:--:-- --:--:--  6220
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.
+ pip install -q cupy-cuda100  chainer 
+ set +ex
Installation succeeded!


In [4]:
 !nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [5]:
# Author: Chris Moody <chrisemoody@gmail.com>
# License: MIT

# This simple example loads the newsgroups data from sklearn
# and train an LDA-like model on it
import os
import os.path
import pickle
import time
import shelve

import chainer
from chainer import cuda
from chainer import serializers
import chainer.optimizers as O
import numpy as np

from lda2vec import utils
from lda2vec import prepare_topics, print_top_words_per_topic, topic_coherence
from lda2vec import LDA2Vec

gpu_id = int(os.getenv('CUDA_GPU', 0))
cuda.get_device(gpu_id).use()
print("Using GPU " + str(gpu_id))

Using GPU 0


In [6]:
print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)

GPU availability: True
cuDNN availablility: True


In [0]:
data_dir = os.getenv('data_dir', './drive/My Drive')
fn_vocab = '{data_dir:s}/vocab.pkl'.format(data_dir=data_dir)
fn_corpus = '{data_dir:s}/corpus.pkl'.format(data_dir=data_dir)
fn_flatnd = '{data_dir:s}/flattened.npy'.format(data_dir=data_dir)
fn_docids = '{data_dir:s}/doc_ids.npy'.format(data_dir=data_dir)
fn_vectors = '{data_dir:s}/vectors.npy'.format(data_dir=data_dir)
with open(fn_vocab, 'rb') as handle:
    vocab = pickle.load(handle)
with open(fn_corpus, 'rb') as handle:
    corpus = pickle.load(handle)
flattened = np.load(fn_flatnd)
doc_ids = np.load(fn_docids)
vectors = np.load(fn_vectors)

In [0]:
# Model Parameters
# Number of documents
n_docs = doc_ids.max() + 1
# Number of unique words in the vocabulary
n_vocab = flattened.max() + 1
# 'Strength' of the dircihlet prior;
clambda = 5.0
# Number of topics to fit
n_topics = 35
batchsize = 16384
# Power for neg sampling
power = float(os.getenv('power', 0.75))
# Intialize with pretrained word vectors
pretrained = bool(int(os.getenv('pretrained', True)))
# Sampling temperature
temperature = float(os.getenv('temperature', 1.0))
# Number of dimensions in a single word vector
n_units = int(os.getenv('n_units', 300))
# Get the string representation for every compact key
words = corpus.word_list(vocab)[:n_vocab]
# How many tokens are in each document
doc_idx, lengths = np.unique(doc_ids, return_counts=True)
doc_lengths = np.zeros(doc_ids.max() + 1, dtype='int32')
doc_lengths[doc_idx] = lengths
# Count all token frequencies
tok_idx, freq = np.unique(flattened, return_counts=True)
term_frequency = np.zeros(n_vocab, dtype='int32')
term_frequency[tok_idx] = freq

In [9]:
for key in sorted(locals().keys()):
    val = locals()[key]
    if len(str(val)) < 100 and '<' not in str(val):
        print(key, val)

Out {}
_ 
__ 
___ 
__doc__ Automatically created module for IPython interactive environment
__loader__ None
__name__ __main__
__package__ None
__spec__ None
_dh ['/content']
_exit_code 0
_i2 from google.colab import drive
drive.mount('/content/drive')
_i3 !curl https://colab.chainer.org/install | sh -
_i4  !nvcc --version
_oh {}
batchsize 16384
clambda 5.0
data_dir ./drive/My Drive
doc_ids [   0    0    0 ... 2466 2466 2466]
doc_idx [   0    1    2 ... 2464 2465 2466]
doc_lengths [1377 1007 1608 ...  797 1215  480]
flattened [  50  545   19 ... 3631   76   49]
fn_corpus ./drive/My Drive/corpus.pkl
fn_docids ./drive/My Drive/doc_ids.npy
fn_flatnd ./drive/My Drive/flattened.npy
fn_vectors ./drive/My Drive/vectors.npy
fn_vocab ./drive/My Drive/vocab.pkl
freq [21271 19806 19320 ...     2     2     1]
gpu_id 0
lengths [1377 1007 1608 ...  797 1215  480]
n_docs 2467
n_topics 35
n_units 300
n_vocab 33595
power 0.75
pretrained True
temperature 1.0
term_frequency [0 0 0 ... 2 2 1]
tok_idx [    

In [10]:
model = LDA2Vec(n_documents=n_docs, n_document_topics=n_topics,
                n_units=n_units, n_vocab=n_vocab, counts=term_frequency,
                n_samples=15, power=power, temperature=temperature)

if os.path.exists('lda2vec.hdf5'):
    print("Reloading from saved")
    serializers.load_hdf5("lda2vec.hdf5", model)
    
if pretrained:
    model.sampler.W.data[:, :] = vectors[:n_vocab, :]




model.to_gpu(gpu_id)
optimizer = O.Adam(final_lr=0.01)
optimizer.setup(model)
clip = chainer.optimizer.GradientClipping(5.0)
optimizer.add_hook(clip)

j = 0
epoch = 0
fraction = batchsize * 1.0 / flattened.shape[0]
progress = shelve.open('progress.shelve')



for epoch in range(200):
    data = prepare_topics(cuda.to_cpu(model.mixture.weights.W.data).copy(),
                          cuda.to_cpu(model.mixture.factors.W.data).copy(),
                          cuda.to_cpu(model.sampler.W.data).copy(),
                          words)
    top_words = print_top_words_per_topic(data)
    # if j % 100 == 0 and j > 100:
    # coherence = topic_coherence(top_words)
    # for k in range(n_topics):
    #     print(k, coherence[(k, 'cv')])
    # kw = dict(top_words=top_words, coherence=coherence, epoch=epoch)
    # progress[str(epoch)] = pickle.dumps(kw)

    data['doc_lengths'] = doc_lengths
    data['term_frequency'] = term_frequency
    np.savez('topics.pyldavis', **data)
    for d, f in utils.chunks(batchsize, doc_ids, flattened):
        t0 = time.time()
        model.cleargrads()
        #optimizer.use_cleargrads(use=False)
        l = model.fit_partial(d.copy(), f.copy())
        # print("after partial fitting:", l)
        prior = model.prior()
        loss = prior * fraction
        loss.backward()
        optimizer.update()
        msg = ("Batch:{j:05d} - Epoch:{epoch:05d} - L:{loss:1.3e} ")
        prior.to_cpu()
        loss.to_cpu()
        t1 = time.time()
        dt = t1 - t0
        rate = batchsize / dt
        # logs = dict(loss=float(l), epoch=epoch, j=j,
        #             prior=float(prior.data), rate=rate)
        
        if j%10==0:
          logs = dict(loss=float(l), epoch=epoch, j=j)
          print(msg.format(**logs))
        j += 1
    serializers.save_hdf5("lda2vec.hdf5", model)

Top words in topic 0 citizenship uranium royalty username ancestry blindness mineral visa hydrothermal rower
Top words in topic 1 seatbelt cyberbullying superconducting texting aortic actuator collider conductive lexicographer ion
Top words in topic 2 wittgenstein mozart fluctuation bycatch clutter insanity aria rearranging sufficiency antimony
Top words in topic 3 alto mitzvah giffords erez civility wept falafel walkability tiananmen lemonade
Top words in topic 4 watermark dodger paternity invoked et loom quota fog phnom headlight
Top words in topic 5 franc pituitary adobe rehydration gamma dopamine confiscated buying metabolism sexting
Top words in topic 6 enceladus kaaba malé ahmadinejad centrifuge merckx assad headline blocker rider
Top words in topic 7 tuscaloosa felony proscenium pulley aerodynamics tidying homer zipline visa interviewing
Top words in topic 8 giffords bloomberg sotomayor tamer runoff hurricane humberto southernmost redistribution tumble
Top words in topic 9 amygd

In [11]:
! cp ./* drive/My\ Drive

cp: -r not specified; omitting directory './drive'
cp: -r not specified; omitting directory './sample_data'
cp: -r not specified; omitting directory './drive'
cp: -r not specified; omitting directory './sample_data'
